# Directory

```bash
├── data
│   ├── out_data
│   ├── rf_data
│   ├── water_data
│   ├── test_data_imp_0831.csv
│   ├── train_data_imp_0831.csv
├── out
├── Imputaton-Submission.ipynb
└── Project-Submission.ipynb
``` 

# OS
- Ubuntu
- Intel i9 10900
- NVIDIA GeForce RTX 3080
- Memory 64GB

# 주의사항

- 메모리 최소 30GB 이상 필요

In [1]:
import pandas as pd
import numpy as np

from glob import glob
from tqdm import tqdm
from scipy import interpolate

#visualization
import matplotlib as mpl
import matplotlib.pylab as plb
import matplotlib.pyplot as plt
import sklearn as sk
import seaborn as sns

import sys
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import Lasso, Ridge
from datetime import datetime, timedelta

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.multioutput import RegressorChain
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor , AdaBoostRegressor, StackingRegressor, BaggingRegressor, VotingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import PolynomialFeatures
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.base import clone

import random
import os
import zipfile
import warnings
warnings.filterwarnings('ignore')


# ANN module
import torch
from torch import nn, optim                           # torch 에서 제공하는 신경망 기술, 손실함수, 최적화를 할 수 있는 함수들을 불러온다.
from torch.utils.data import DataLoader, Dataset      # 데이터를 모델에 사용할 수 있게 정리해주는 라이브러리.
import torch.nn.functional as F                       # torch 내의 세부적인 기능을 불러옴.

import datetime

In [2]:
def fft_freq(x, number):
    s_fft = np.fft.fft(x) # 추후 IFFT를 위해 abs를 취하지 않은 값을 저장한다.
    amplitude = abs(s_fft)*(2/len(s_fft))
    frequency = np.fft.fftfreq(len(s_fft))
    fft_freq = frequency.copy()    
    peak_index = amplitude[:int(len(amplitude)/2)].argsort()[-number:]
    peak_freq = fft_freq[peak_index]
    
    return peak_freq

In [3]:
def fft_amp(x, number):
    s_fft = np.fft.fft(x) # 추후 IFFT를 위해 abs를 취하지 않은 값을 저장한다.
    amplitude = abs(s_fft)*(2/len(s_fft))
    frequency = np.fft.fftfreq(len(s_fft))
    fft_freq = frequency.copy()    
    peak_index = amplitude[:int(len(amplitude)/2)].argsort()[-number:]
    peak_amp = amplitude[peak_index]
    
    return peak_amp

In [4]:
SEED = 123
def seed_everything(seed=123):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
seed_everything(SEED)

In [5]:
train_data = pd.read_csv("./data/train_data_imp_0831.csv")
test_data = pd.read_csv("./data/test_data_imp_0831.csv")

In [6]:
y_columns = ['wl_1019630' ,  'wl_1018683'  ,  'wl_1018680'  ,  'wl_1018662']

In [7]:
train_data['month'] = train_data['ymdhm'].apply(lambda x: np.int(datetime.datetime.strptime(x, '%Y-%m-%d %H:%M').strftime('%m')))

In [8]:
test_data['month'] = test_data['ymdhm'].apply(lambda x: np.int(datetime.datetime.strptime(x, '%Y-%m-%d %H:%M').strftime('%m')))

## 변수 선택 및 최적 lag 선택

In [9]:
class Model():
    def __init__(self, data, lag, y_column, x_column, shift):
        self.data = data.copy()
        self.lag = lag
        self.y_cols = y_column
        self.x_cols = x_column
        self.shift = shift
        
    def train(self, estimator, scale, square):
        self.square = square

        self.scale = scale
        if self.scale == True:
            self.scaler = StandardScaler()
            self.scaler.fit(self.data[self.x_cols])
            self.data[self.x_cols] = self.scaler.transform(self.data[self.x_cols])

            
        col_list = ['month']
        for x_col in self.x_cols:
            self.data[x_col+'_percentage_lag1']= (self.data[x_col].shift(1) - self.data[x_col].shift(2))/(self.data[x_col].shift(2)+1e-6)
            col_list.append(x_col+'_percentage_lag1')
            self.data[x_col+'_percentage_lag2']= (self.data[x_col].shift(2) - self.data[x_col].shift(3))/(self.data[x_col].shift(3)+1e-6)
            col_list.append(x_col+'_percentage_lag2')
        
#         print(self.data.columns)
        
        for x_col in self.x_cols:
            for s in range(1, self.lag+1):
                self.data[x_col+'_lag_'+str(s)] = self.data[x_col].shift(s)
                self.data['year_lag_'+str(s)] = self.data['year'].shift(s)
                col_list.append(x_col+'_lag_'+str(s))
#         print(col_list)   
#         print(self.data.columns)
        Train_data = self.data[self.data['year'] == self.data['year_lag_'+str(s)]]
        Train_data = Train_data[['ymdhm', 'year']+col_list + self.y_cols].dropna()
        Train_data = Train_data.reset_index(drop=True)

        for x in tqdm(self.x_cols):

            ts= Train_data.filter(regex=x+'_lag')
#             print(ts.columns)

            number = np.int(self.lag/3)
            fft_fr = ts.apply(lambda x: fft_freq(np.array(x), number)  ,axis=1)
            fft_mp = ts.apply(lambda x: fft_amp(np.array(x), number)  ,axis=1)
            
            numbers = range(number)
            for num in numbers:
                Train_data['freq_'+ x + '_'+str(num)]= fft_fr.apply(lambda x: x[num])
                Train_data['amp_'+ x + '_'+str(num)]= fft_mp.apply(lambda x: x[num])

                col_list.append('freq_'+ x + '_' +str(num))
                col_list.append('amp_'+ x + '_' +str(num))
        #     print(ts.columns)
            square_columns = ts.columns
            for sc in square_columns:
                Train_data['square_'+ sc] = Train_data[sc] ** 2
                col_list.append('square_'+ sc )

#         print(Train_data.columns)
        
        train_X = Train_data[col_list]
        train_Y = Train_data[self.y_cols]
        
        print("X shape: ", train_X.shape, "Y shape: ", train_Y.shape)
        
        self.model_list = []
        kf = KFold(n_splits=3, shuffle = True, random_state = 1234)
        
        mse_list = []
        for train_index, val_index in kf.split(train_X.index):
            x_train = train_X.iloc[train_index]
            x_valid = train_X.iloc[val_index]
            y_train = train_Y.iloc[train_index]
            y_valid = train_Y.iloc[val_index]
            
            models = clone(estimator)
            
            models.fit(x_train, y_train)
            preds = models.predict(x_valid)
            mse = mean_squared_error(np.array(y_valid),preds)**0.5
            mse_list.append(mse)
            
            self.model_list.append(models) 
        
        self.data = []
        
        return np.array(mse_list).mean()

In [10]:
estimator = RegressorChain( LinearRegression())

In [11]:
cols = train_data.columns
x_columns = list(cols.drop(['ymdhm', 'time', 'year', 'wl_1018662', 'fw_1018662', 'wl_1018680', 'wl_1018683', 'fw_1018683', 'wl_1019630', 'fw_1019630', 'fw_out_1018680', 'fw_out_1019675', 'wl_out_1018658', 'fw_out_1018658', 'month']))
drop_cols = x_columns.copy()

In [12]:
best_mse = 10
init_drop = ['ymdhm', 'year', 'time', 'wl_1018662', 'fw_1018662', 'wl_1018680', 'wl_1018683', 'fw_1018683', 'wl_1019630', 'fw_1019630', 'fw_out_1018680', 'fw_out_1019675', 'wl_out_1018658', 'fw_out_1018658', 'month']
start_lag = 12
end_lag = 24
lag_list = list(range(start_lag,end_lag+1, 6))
for i in range(len(drop_cols)):
    
    x_columns = list(cols.drop(init_drop + [drop_cols[i]]))
    print("drop cols: ", init_drop + [drop_cols[i]])
    model_list = []
    mse_list = []
    for l in lag_list:    
        model = Model(train_data, l, y_columns, x_columns, shift = 0)
        # model.Imputation(shift = 3)
        MSE=model.train(estimator, scale = True, square =True)
        print("lag ", l, ": ", MSE)
        mse_list.append(MSE)
        model_list.append(model)
    print("best lag: ", lag_list[np.argmin(mse_list)], "MSE: ", mse_list[np.argmin(mse_list)])
    
    if best_mse > mse_list[np.argmin(mse_list)]:
        best_mse = mse_list[np.argmin(mse_list)]
        best_model = model_list[np.argmin(mse_list)]
        init_drop = init_drop + [drop_cols[i]]
        best_lag = lag_list[np.argmin(mse_list)]
        print("drop!")
    

drop cols:  ['ymdhm', 'year', 'time', 'wl_1018662', 'fw_1018662', 'wl_1018680', 'wl_1018683', 'fw_1018683', 'wl_1019630', 'fw_1019630', 'fw_out_1018680', 'fw_out_1019675', 'wl_out_1018658', 'fw_out_1018658', 'month', 'swl']


100%|██████████| 25/25 [05:37<00:00, 13.48s/it]


X shape:  (267853, 851) Y shape:  (267853, 4)
lag  12 :  1.1265182466735775


100%|██████████| 25/25 [05:36<00:00, 13.47s/it]


X shape:  (267703, 1251) Y shape:  (267703, 4)
lag  18 :  1.1240537046229346


100%|██████████| 25/25 [05:41<00:00, 13.66s/it]


X shape:  (267553, 1651) Y shape:  (267553, 4)
lag  24 :  1.139241349321898
best lag:  18 MSE:  1.1240537046229346
drop!
drop cols:  ['ymdhm', 'year', 'time', 'wl_1018662', 'fw_1018662', 'wl_1018680', 'wl_1018683', 'fw_1018683', 'wl_1019630', 'fw_1019630', 'fw_out_1018680', 'fw_out_1019675', 'wl_out_1018658', 'fw_out_1018658', 'month', 'swl', 'inf']


  4%|▍         | 1/24 [00:18<06:56, 18.09s/it]


KeyboardInterrupt: 

In [13]:
x_columns = list(cols.drop(init_drop))
x_columns

['inf',
 'sfw',
 'ecpc',
 'tototf',
 'tide_level',
 'rf_10184100',
 'rf_10184110',
 'rf_10184140',
 'rf_out_0194010',
 'fw_out_1018610',
 'wl_out_1018610',
 'fw_out_1018640',
 'wl_out_1018640',
 'fw_out_1018662',
 'wl_out_1018662',
 'fw_out_1018675',
 'wl_out_1018675',
 'wl_out_1018680',
 'fw_out_1018683',
 'wl_out_1018683',
 'fw_out_1018697',
 'wl_out_1018697',
 'fw_out_1019630',
 'wl_out_1019630',
 'wl_out_1019675']

In [14]:
model_list = []

In [15]:
tt = train_data.copy()
scaler = StandardScaler()
scaler.fit(tt[x_columns])
tt[x_columns] = scaler.transform(tt[x_columns])
col_list = ['month']
for x_col in x_columns:
    tt[x_col+'_percentage_lag1_diff1']= (tt[x_col].shift(1) - tt[x_col].shift(2))/(tt[x_col].shift(2)+1e-6)
    col_list.append(x_col+'_percentage_lag1_diff1')
    tt[x_col+'_percentage_lag1_diff2']= (tt[x_col].shift(1) - tt[x_col].shift(3))/(tt[x_col].shift(3)+1e-6)
    col_list.append(x_col+'_percentage_lag1_diff2')
    tt[x_col+'_percentage_lag2']= (tt[x_col].shift(2) - tt[x_col].shift(3))/(tt[x_col].shift(3)+1e-6)
    col_list.append(x_col+'_percentage_lag2')
    tt[x_col+'_percentage_lag3']= (tt[x_col].shift(3) - tt[x_col].shift(4))/(tt[x_col].shift(4)+1e-6)
    col_list.append(x_col+'_percentage_lag3')

for x_col in x_columns:
    tt[x_col+'_diff1']= (tt[x_col].shift(1) - tt[x_col].shift(2))
    col_list.append(x_col+'_diff1')
    tt[x_col+'_diff2']= (tt[x_col].shift(1) - tt[x_col].shift(3))
    col_list.append(x_col+'_diff2')
    
for x_col in x_columns:
    tt[x_col+'diff6']= tt[x_col].shift(1) - tt[x_col].shift(7)
    col_list.append(x_col+'diff6')
    tt[x_col+'diff12']= tt[x_col].shift(1) - tt[x_col].shift(13)
    col_list.append(x_col+'diff12')   
    
for x_col in x_columns:
    tt[x_col+'_percentage_diff6']= (tt[x_col].shift(1) - tt[x_col].shift(7))/(tt[x_col].shift(7)+1e-6)
    col_list.append(x_col+'_percentage_diff6')
    tt[x_col+'_percentage_diff12']= (tt[x_col].shift(1) - tt[x_col].shift(13))/(tt[x_col].shift(13)+1e-6)
    col_list.append(x_col+'_percentage_diff12')


    
for x_col in x_columns:
    tt[x_col+'_max']= np.max(np.array([tt[x_col].shift(i) for i in range(1, 7)]), axis =0)
    col_list.append(x_col+'_max')
    tt[x_col+'_min']= np.min(np.array([tt[x_col].shift(i) for i in range(1, 7)]), axis =0)
    col_list.append(x_col+'_min')
    
print(tt.columns)




for x_col in x_columns:
    for s in range(1, best_lag+1):
        tt[x_col+'_lag_'+str(s)] = tt[x_col].shift(s)
        tt['year_lag_'+str(s)] = tt['year'].shift(s)
        col_list.append(x_col+'_lag_'+str(s))
#         print(col_list)   
print(tt.columns)
tt = tt[tt['year'] == tt['year_lag_'+str(s)]]
tt = tt[['ymdhm', 'year']+col_list + y_columns].dropna()
tt = tt.reset_index(drop=True)



for x in tqdm(x_columns):
    ts= tt.filter(regex=x+'_lag')
    number = np.int(best_lag/2)
    fft_fr = ts.apply(lambda x: fft_freq(np.array(x), number)  ,axis=1)
    fft_mp = ts.apply(lambda x: fft_amp(np.array(x), number)  ,axis=1)

    numbers = range(number)
    for num in numbers:
        tt['freq_'+ x + '_'+str(num)]= fft_fr.apply(lambda x: x[num])
        tt['amp_'+ x + '_'+str(num)]= fft_mp.apply(lambda x: x[num])

        col_list.append('freq_'+ x + '_' +str(num))
        col_list.append('amp_'+ x + '_' +str(num))
#     print(ts.columns)
    square_columns = ts.columns
    for sc in square_columns:
        tt['square_'+ sc] = tt[sc] ** 2
        col_list.append('square_'+ sc )
        
    root_columns = ts.columns
    for rc in root_columns:
        tt['root_'+ rc] = np.abs(tt[rc]) ** 0.5
        col_list.append('root_'+ rc )
        

Index(['ymdhm', 'swl', 'inf', 'sfw', 'ecpc', 'tototf', 'tide_level',
       'wl_1018662', 'fw_1018662', 'wl_1018680',
       ...
       'fw_out_1018697_max', 'fw_out_1018697_min', 'wl_out_1018697_max',
       'wl_out_1018697_min', 'fw_out_1019630_max', 'fw_out_1019630_min',
       'wl_out_1019630_max', 'wl_out_1019630_min', 'wl_out_1019675_max',
       'wl_out_1019675_min'],
      dtype='object', length=341)
Index(['ymdhm', 'swl', 'inf', 'sfw', 'ecpc', 'tototf', 'tide_level',
       'wl_1018662', 'fw_1018662', 'wl_1018680',
       ...
       'wl_out_1019675_lag_9', 'wl_out_1019675_lag_10',
       'wl_out_1019675_lag_11', 'wl_out_1019675_lag_12',
       'wl_out_1019675_lag_13', 'wl_out_1019675_lag_14',
       'wl_out_1019675_lag_15', 'wl_out_1019675_lag_16',
       'wl_out_1019675_lag_17', 'wl_out_1019675_lag_18'],
      dtype='object', length=809)


  0%|          | 0/25 [00:11<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
train_X = tt[col_list]
train_Y = tt[y_columns]

In [ ]:
best_lag

In [ ]:
train_X

## 모형 적합

In [ ]:
model_list = []
kf = KFold(n_splits=5, shuffle = True, random_state = 1234)
for train_index, val_index in kf.split(train_X.index):
    x_train = train_X.iloc[train_index]
    x_valid = train_X.iloc[val_index]
    y_train = train_Y.iloc[train_index]
    y_valid = train_Y.iloc[val_index]
    models =  [RegressorChain(LGBMRegressor(n_estimators=5000)),
               RegressorChain(LinearRegression())]



    for k in range(len(models)):
        models[k].fit(x_train, y_train)
    preds = models[0].predict(x_valid)
    for k in range(1,len(models)):
        preds += models[k].predict(x_valid).reshape(-1,4)
    preds = preds/(k+1)
    print("MSE: ", mean_squared_error(np.array(y_valid),preds)**0.5)
    for i in range(4):
        print(mean_squared_error(np.array(y_valid)[:,i], preds[:,i])**0.5)
        plt.scatter(np.array(y_valid)[:,i], preds[:,i])
        plt.show()
    
    model_list.append(models)    

In [ ]:
cnt = 0
mse_list = []
for train_index, val_index in kf.split(train_X.index):    
    x_valid = train_X.iloc[val_index]
    y_valid = train_Y.iloc[val_index]
    models = model_list[cnt]
    mses = []
    for k in range(len(models)):
        preds = models[k].predict(x_valid)
        mse = [mean_squared_error(np.array(y_valid)[:,i], preds[:,i])**0.5 for i in range(4)]
        print(models[k], mse)
        mse = [1/mean_squared_error(np.array(y_valid)[:,i], preds[:,i])**0.5 for i in range(4)]
        mses.append(mse)
    mse_list.append(mses)
    cnt += 1
    

In [ ]:
te = pd.concat([train_data.iloc[-(best_lag):], test_data])
te[x_columns] = scaler.transform(te[x_columns])

In [ ]:
x_columns

In [ ]:
col_list = ['month']

for x_col in x_columns:
    te[x_col+'_percentage_lag1_diff1']= (te[x_col].shift(1) - te[x_col].shift(2))/(te[x_col].shift(2)+1e-6)
    col_list.append(x_col+'_percentage_lag1_diff1')
    te[x_col+'_percentage_lag1_diff2']= (te[x_col].shift(1) - te[x_col].shift(3))/(te[x_col].shift(3)+1e-6)
    col_list.append(x_col+'_percentage_lag1_diff2')
    te[x_col+'_percentage_lag2']= (te[x_col].shift(2) - te[x_col].shift(3))/(te[x_col].shift(3)+1e-6)
    col_list.append(x_col+'_percentage_lag2')
    te[x_col+'_percentage_lag3']= (te[x_col].shift(3) - te[x_col].shift(4))/(te[x_col].shift(4)+1e-6)
    col_list.append(x_col+'_percentage_lag3')

for x_col in x_columns:
    te[x_col+'_diff1']= (te[x_col].shift(1) - te[x_col].shift(2))
    col_list.append(x_col+'_diff1')
    te[x_col+'_diff2']= (te[x_col].shift(1) - te[x_col].shift(3))
    col_list.append(x_col+'_diff2')
    
for x_col in x_columns:
    te[x_col+'diff6']= te[x_col].shift(1) - te[x_col].shift(7)
    col_list.append(x_col+'diff6')
    te[x_col+'diff12']= te[x_col].shift(1) - te[x_col].shift(13)
    col_list.append(x_col+'diff12')   
    
for x_col in x_columns:
    te[x_col+'_percentage_diff6']= (te[x_col].shift(1) - te[x_col].shift(7))/(te[x_col].shift(7)+1e-6)
    col_list.append(x_col+'_percentage_diff6')
    te[x_col+'_percentage_diff12']= (te[x_col].shift(1) - te[x_col].shift(13))/(te[x_col].shift(13)+1e-6)
    col_list.append(x_col+'_percentage_diff12')

    

    
for x_col in x_columns:
    te[x_col+'_max']= np.max(np.array([te[x_col].shift(i) for i in range(1, 7)]), axis =0)
    col_list.append(x_col+'_max')
    te[x_col+'_min']= np.min(np.array([te[x_col].shift(i) for i in range(1, 7)]), axis =0)
    col_list.append(x_col+'_min')
    
print(te.columns)

for x_col in x_columns:
    for s in range(1, best_lag+1):
        te[x_col+'_lag_'+str(s)] = te[x_col].shift(s)
        te['year_lag_'+str(s)] = te['year'].shift(s)
        col_list.append(x_col+'_lag_'+str(s))
#         print(col_list)   
print(te.columns)
te = te[te['year'] == te['year_lag_'+str(s)]]
te = te[['ymdhm', 'year']+col_list + y_columns].dropna()
te = te.reset_index(drop=True)


for x in tqdm(x_columns):

    ts= te.filter(regex=x+'_lag')
    number = np.int(best_lag/2)
    fft_fr = ts.apply(lambda x: fft_freq(np.array(x), number)  ,axis=1)
    fft_mp = ts.apply(lambda x: fft_amp(np.array(x), number)  ,axis=1)

    numbers = range(number)
    for num in numbers:
        te['freq_'+ x + '_'+str(num)]= fft_fr.apply(lambda x: x[num])
        te['amp_'+ x + '_'+str(num)]= fft_mp.apply(lambda x: x[num])

        col_list.append('freq_'+ x + '_' +str(num))
        col_list.append('amp_'+ x + '_' +str(num))
#     print(ts.columns)
    square_columns = ts.columns
    print(square_columns)
    for sc in square_columns:
        te['square_'+ sc] = te[sc] ** 2
        col_list.append('square_'+ sc )
        
    root_columns = ts.columns
    for rc in root_columns:
        te['root_'+ rc] = np.abs(te[rc]) ** 0.5
        col_list.append('root_'+ rc )
        

In [ ]:
list(train_X.columns)

In [ ]:
list(test_X.columns)

In [ ]:

y_columns = [ 'wl_1019630' ,  'wl_1018683'  ,  'wl_1018680'  ,  'wl_1018662']
test = te[col_list+y_columns].dropna()
test_X = test[col_list]
test_Y = test[y_columns]

In [ ]:
test_Y

In [ ]:
test_X

In [ ]:
test_X.isna().sum()

In [ ]:

preds_list = []
for i in range(len(model_list)):
    models = model_list[i]
    preds_test = models[0].predict(test_X)
    for k in range(1,len(models)):
        preds_test += models[k].predict(test_X).reshape(-1,4)
    preds_test = preds_test/(k+1)  
    preds_list.append(preds_test)
        

In [ ]:
preds_1 = np.array(preds_list).mean(0)
submit = pd.read_csv('./data/sample_submission.csv')
submit[y_columns] = preds_1
submit.to_csv('./out/submit_ensemble_0902.csv', index=False)

In [ ]:
weight_list = np.array(mse_list)

In [ ]:
for i in range(5): # fold
    for k in range(4): # Y
        mm = np.array(mse_list)[i,:,:].sum(0)
        weight_list[i,:,k] = weight_list[i,:,k]/mm[k]
weights = weight_list.mean(0)

In [ ]:
weights

In [ ]:

preds_list = []
for i in range(len(model_list)):
    models = model_list[i]
    preds_test = models[0].predict(test_X) * weights[0,:]
    for j in range(1, len(models)):
        preds_test += models[j].predict(test_X) * weights[j,:]
    
    preds_list.append(preds_test)


In [ ]:
preds_2 = np.array(preds_list).mean(0)
submit = pd.read_csv('./data/sample_submission.csv')
submit[y_columns] = preds_2
submit.to_csv('./out/submit_ensemble_0902 (1).csv', index=False)

In [ ]:
submit = pd.read_csv('./data/sample_submission.csv')
submit[y_columns] = (preds_1+preds_2)/2
submit.to_csv('./out/submit_ensemble_0902_merge.csv', index=False)